In [1]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
#from eda_py import EDA
from eALS_adaptor import eALSAdaptor
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from cv_py import CrossValidation
#from visualize_py import Visualize

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

[Errno 2] No such file or directory: 'C:Users781110104OneDrive - GenpactDocumentsVSCode'
/home/cbarkey


In [53]:
dl = DataLoader()

In [3]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

In [6]:
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [3]:
from scipy import sparse

In [16]:
from scipy import sparse
user_item_csr = sparse.load_npz('user_item_csr.npz')
user_item_csr_t = sparse.load_npz('user_item_csr_t.npz')

In [64]:
userf_t = sparse.load_npz('userf_t.npz')
itemf_t = sparse.load_npz('itemf_t.npz')

In [17]:
# initial split
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [5]:
cv = CrossValidation(5)

In [19]:
validate, calibrate = cv.split_k_fold(train, 22)
validate_t, calibrate_t = cv.split_k_fold(train_t, 22)

In [22]:
space_iALS = {'factors' : [64], 'regularization' : [80, 90, 100, 110, 120, 130, 140, 150, 160, 170], 'alpha' : [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], 'iterations' : [15]}
hyper_ials = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')

In [23]:
space_iALS_t = {'factors' : [64], 'regularization' : [40, 50, 60, 70, 80], 'alpha' : [0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5], 'iterations' : [15]}
hyper_ials_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')

In [25]:
space_eALS_t = {'factors' : [64], 'regularization' : [40, 50, 60, 70], 'alpha' : [0.001, 0.05, 0.1, 0.2], 
'w0' : [1000, 2000, 3000, 4000, 5000], 'iterations' : [15]}
hyper_eals_te = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_eALS_t, model_class='eALS')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [28]:
hyper_eals_te.sort_values('precision', ascending=False)

,factors,regularization,alpha,w0,iterations,precision,map,ndcg,auc,mpr
47,64,60,0.050,3000,15,0.5655,0.449721,0.587544,0.516083,0.121139
79,64,70,0.200,5000,15,0.5650,0.454109,0.589647,0.515824,0.121043
73,64,70,0.100,4000,15,0.5650,0.451516,0.589567,0.516008,0.121362
52,64,60,0.100,3000,15,0.5645,0.450064,0.586940,0.515890,0.120869
31,64,50,0.100,2000,15,0.5640,0.448368,0.587431,0.516027,0.120116
...,...,...,...,...,...,...,...,...,...,...
55,64,60,0.200,1000,15,0.5270,0.411156,0.555466,0.514872,0.119048
65,64,70,0.050,1000,15,0.5185,0.401283,0.546033,0.514376,0.120375
70,64,70,0.100,1000,15,0.5185,0.401744,0.546336,0.514340,0.120203
60,64,70,0.001,1000,15,0.5180,0.400447,0.545374,0.514448,0.120555


In [59]:
space_eALS = {'factors' : [64], 'regularization' : [200, 230, 250, 270, 290], 'alpha' : [0.01, 0.05, 0.1, 0.15, 0.2], 
'w0' : [25000, 30000, 35000, 40000], 'iterations' : [15]}
hyper_eals_t = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_eALS, model_class='eALS')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [14]:
space_eALS = {'factors' : [64], 'regularization' : [200, 230, 250], 'alpha' : [0.1, 0.15, 0.2], 
'w0' : [55000, 60000, 65000], 'iterations' : [15]}
hyper_eals_t_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_eALS, model_class='eALS')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [70]:
hyper_eals_t = pd.concat([hyper_eals_t, hyper_eals_t_add], axis=0)

In [71]:
hyper_eals_t.sort_values('precision', ascending=False).head(10)

,factors,regularization,alpha,w0,iterations,precision,map,ndcg,auc,mpr
9,64,230,0.15,60000,15,0.862244,0.811255,0.873468,0.505549,0.086239
8,64,230,0.15,55000,15,0.861980,0.811025,0.873349,0.505552,0.085497
17,64,250,0.20,60000,15,0.861914,0.810458,0.872716,0.505539,0.085226
7,64,230,0.10,60000,15,0.861914,0.811410,0.873637,0.505565,0.085907
23,64,230,0.15,50000,15,0.861914,0.811036,0.873404,0.505558,0.084769
12,64,250,0.10,55000,15,0.861716,0.811190,0.873361,0.505557,0.084342
16,64,250,0.20,55000,15,0.861584,0.810239,0.872434,0.505534,0.084624
13,64,250,0.10,60000,15,0.861584,0.811391,0.873433,0.505560,0.084805
21,64,230,0.10,50000,15,0.861518,0.811371,0.873347,0.505549,0.084590
14,64,250,0.15,55000,15,0.861452,0.810798,0.872679,0.505539,0.084423


In [24]:
space_BPR = {'factors' : [64], 'regularization' : [0.005, 0.01, 0.02, 0.03, 0.04], 'learning_rate' : [0.003, 0.005, 0.007, 0.009, 0.01, 0.02, 0.03], 'iterations' : [15]}
hyper_bpr_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_BPR, model_class='BPR')

In [11]:
space_BPR_t = {'factors' : [64], 'regularization' : [0.035, 0.04, 0.045], 'learning_rate' : [0.025, 0.03, 0.035], 'iterations' : [15]}
hyper_bpr_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_BPR_t, model_class='BPR')

In [12]:
hyper_bpr_t.sort_values('precision', ascending=False).head(10)

,factors,regularization,learning_rate,iterations,precision,map,ndcg,auc,mpr
4,64,0.040,0.030,15,0.3420,0.228229,0.369557,0.508651,0.137628
5,64,0.040,0.035,15,0.3420,0.222744,0.367403,0.508845,0.124612
1,64,0.035,0.030,15,0.3415,0.225769,0.368148,0.508605,0.131905
8,64,0.045,0.035,15,0.3415,0.223398,0.367526,0.508893,0.129453
7,64,0.045,0.030,15,0.3385,0.228882,0.368824,0.508439,0.143474
2,64,0.035,0.035,15,0.3360,0.217862,0.360397,0.508716,0.120275
0,64,0.035,0.025,15,0.3325,0.226294,0.362766,0.508231,0.147016
3,64,0.040,0.025,15,0.3060,0.209928,0.335642,0.507213,0.153924
6,64,0.045,0.025,15,0.2820,0.193435,0.311675,0.506353,0.168839


In [25]:
space_LMF = {'factors' : [64], 'regularization' : [10, 20, 30, 40, 50], 'learning_rate' : [0.8, 1.0, 1.2, 1.4], 
'iterations' : [15], 'neg_prop': [40, 50, 60, 70]}
hyper_lmf_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_LMF, model_class='LMF')

In [13]:
space_LMF_t = {'factors' : [64], 'regularization' : [10, 20, 30], 'learning_rate' : [1.0, 1.2, 1.4], 
'iterations' : [15], 'neg_prop': [0, 5, 10]}
hyper_lmf_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_LMF_t, model_class='LMF')

In [15]:
hyper_lmf_t.sort_values('precision', ascending=False).head(10)

,factors,regularization,learning_rate,iterations,neg_prop,precision,map,ndcg,auc,mpr
12,64,20,1.2,15,0,0.4460,0.315966,0.460762,0.512116,0.215903
15,64,20,1.4,15,0,0.4205,0.290802,0.434689,0.511122,0.226015
9,64,20,1.0,15,0,0.4150,0.280764,0.418803,0.511160,0.205402
18,64,30,1.0,15,0,0.2950,0.161067,0.285824,0.507510,0.253996
21,64,30,1.2,15,0,0.2800,0.167777,0.276519,0.506680,0.273009
6,64,10,1.4,15,0,0.2410,0.127951,0.244017,0.506109,0.175954
0,64,10,1.0,15,0,0.2250,0.112675,0.225172,0.505608,0.172340
24,64,30,1.4,15,0,0.2085,0.120608,0.209929,0.504851,0.287549
3,64,10,1.2,15,0,0.2055,0.098568,0.202344,0.505182,0.174616
7,64,10,1.4,15,5,0.0505,0.017047,0.050575,0.501138,0.407972


In [45]:
interactions_csr = sparse.load_npz('interactions_csr_t.npz')
weights_csr = sparse.load_npz('weights_csr_t.npz')
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)
val_w, cal_w = cv.split_k_fold(train_w, 22)
val_i, cal_i = cv.split_k_fold(train_i, 22)

In [6]:
interactions_csr = sparse.load_npz('interactions_csr.npz')
weights_csr = sparse.load_npz('weights_csr.npz')
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)
val_w, cal_w = cv.split_k_fold(train_w, 22)
val_i, cal_i = cv.split_k_fold(train_i, 22)

In [7]:
itemf = sparse.load_npz('itemf.npz')

In [56]:
space_FM_warp_t = {'factors' : [64], 'regularization' : [0.0001, 0.0005, 0.001], 'learning_rate' : [0.001, 0.005, 0.01], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [1, 2, 3]}
hyper_fm_t = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp_t, model_class='FM', user_features=None)

In [97]:
space_FM_bpr_t = {'factors' : [64], 'regularization' : [0.00000005, 0.0000001, 0.0000005, 0.000001, 0.000005], 'learning_rate' : [0.01, 0.03, 0.05, 0.07, 0.1], 'iterations' : [15], 'loss' : ['logistic'], 'max_sampled' : [1]}
hyper_fm_t = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_bpr_t, model_class='FM', user_features=None)

In [101]:
space_FM_log_t = {'factors' : [64], 'regularization' : [0.00000001, 0.00000005, 0.0000001, 0.0000005], 'learning_rate' : [0.0005, 0.001, 0.005], 'iterations' : [15], 'loss' : ['logistic'], 'max_sampled' : [1]}
hyper_fm_t_uf = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_log_t, model_class='FM', user_features=userf_t)
hyper_fm_t_uf.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
1,64,1.000000e-08,0.0010,15,logistic,1,0.2085,0.131018,0.228186,0.506885,0.332934
4,64,5.000000e-08,0.0010,15,logistic,1,0.2085,0.131018,0.228186,0.506885,0.332934
7,64,1.000000e-07,0.0010,15,logistic,1,0.2085,0.131018,0.228186,0.506885,0.332934
10,64,5.000000e-07,0.0010,15,logistic,1,0.2085,0.130964,0.228166,0.506885,0.332935
0,64,1.000000e-08,0.0005,15,logistic,1,0.2030,0.121112,0.218547,0.506592,0.349871


In [96]:
hyper_fm_t.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
1,64,1.000000e-07,0.03,15,logistic,1,0.5005,0.392098,0.528941,0.513969,0.177610
6,64,5.000000e-07,0.03,15,logistic,1,0.5000,0.391817,0.528637,0.513959,0.177423
11,64,1.000000e-06,0.03,15,logistic,1,0.4995,0.391634,0.528348,0.513941,0.177193
16,64,5.000000e-06,0.03,15,logistic,1,0.4975,0.390683,0.527185,0.513885,0.175460
12,64,1.000000e-06,0.05,15,logistic,1,0.4975,0.390319,0.525943,0.513907,0.200491


In [57]:
hyper_fm_t.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
13,64,0.0005,0.005,15,warp,2,0.3890,0.266485,0.415136,0.510613,0.121995
14,64,0.0005,0.005,15,warp,3,0.3885,0.255578,0.403465,0.510298,0.121467
22,64,0.0010,0.005,15,warp,2,0.3880,0.261744,0.405289,0.510399,0.123734
21,64,0.0010,0.005,15,warp,1,0.3815,0.252923,0.402975,0.509862,0.120874
12,64,0.0005,0.005,15,warp,1,0.3810,0.253972,0.402976,0.509693,0.119682


In [62]:
space_FM_warp_t = {'factors' : [64], 'regularization' : [0.00005, 0.0001, 0.0005], 'learning_rate' : [0.0005, 0.001, 0.005], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [3, 5, 10]}
hyper_fm_t = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp_t, model_class='FM', user_features=userf_t)

In [63]:
hyper_fm_t.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
13,64,0.00010,0.001,15,warp,5,0.4225,0.294706,0.445709,0.511443,0.140160
4,64,0.00005,0.001,15,warp,5,0.4210,0.293868,0.446164,0.511614,0.140249
23,64,0.00050,0.001,15,warp,10,0.4210,0.295630,0.448218,0.511581,0.140721
5,64,0.00005,0.001,15,warp,10,0.4200,0.293430,0.445423,0.511560,0.140721
14,64,0.00010,0.001,15,warp,10,0.4195,0.292390,0.443924,0.511407,0.140704


In [75]:
space_FM_warp_t = {'factors' : [64], 'regularization' : [0.005, 0.01, 0.05], 'learning_rate' : [0.01, 0.05, 0.1], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [20, 30, 40]}
hyper_fm_t = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp_t, model_class='FM', item_features=itemf_t)

In [76]:
hyper_fm_t.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
13,64,0.010,0.05,15,warp,30,0.4040,0.271091,0.422245,0.511786,0.151166
1,64,0.005,0.01,15,warp,30,0.4005,0.272533,0.418572,0.510856,0.114876
2,64,0.005,0.01,15,warp,40,0.3990,0.270630,0.420209,0.510800,0.115137
14,64,0.010,0.05,15,warp,40,0.3955,0.263687,0.419209,0.511752,0.150879
10,64,0.010,0.01,15,warp,30,0.3920,0.267485,0.412675,0.511264,0.123818


In [81]:
space_FM_warp_t = {'factors' : [64], 'regularization' : [0.01, 0.05, 0.1], 'learning_rate' : [0.05, 0.01, 0.05], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [50, 60, 70, 80]}
hyper_fm_t = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp_t, model_class='FM', user_features=userf_t, item_features=itemf_t)

In [82]:
hyper_fm_t.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
17,64,0.05,0.01,15,warp,60,0.4180,0.292206,0.439932,0.511259,0.126214
16,64,0.05,0.01,15,warp,50,0.4145,0.285752,0.433076,0.511295,0.125700
19,64,0.05,0.01,15,warp,80,0.4105,0.292568,0.431627,0.510843,0.126542
18,64,0.05,0.01,15,warp,70,0.4060,0.283131,0.425181,0.510718,0.126119
4,64,0.01,0.01,15,warp,50,0.3765,0.237182,0.386251,0.510198,0.118223


In [8]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0001, 0.0005, 0.001], 'learning_rate' : [0.01, 0.05, 0.1], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [80, 90, 100]}
hyper_fm = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp, model_class='FM', item_features=itemf)

In [9]:
hyper_fm.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
14,64,0.0005,0.05,15,warp,100,0.759802,0.680553,0.770934,0.504381,0.093285
13,64,0.0005,0.05,15,warp,90,0.759736,0.678633,0.771419,0.504403,0.093127
12,64,0.0005,0.05,15,warp,80,0.755248,0.672263,0.765828,0.504363,0.093146
17,64,0.0005,0.10,15,warp,100,0.754257,0.680041,0.766700,0.504243,0.108732
9,64,0.0005,0.01,15,warp,80,0.752673,0.676772,0.764140,0.504356,0.085778


In [10]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0005], 'learning_rate' : [0.05], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [110]}
hyper_fm = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp, model_class='FM', item_features=itemf)

In [11]:
hyper_fm.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
0,64,0.0005,0.05,15,warp,110,0.754785,0.674803,0.766354,0.5043,0.093169


In [12]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0001, 0.0005, 0.001], 'learning_rate' : [0.01, 0.05, 0.1], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [110]}
hyper_fm = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp, model_class='FM', item_features=itemf)

In [13]:
hyper_fm.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
4,64,0.0005,0.05,15,warp,110,0.754785,0.674803,0.766354,0.504300,0.093169
3,64,0.0005,0.01,15,warp,110,0.751551,0.673293,0.762287,0.504320,0.085699
0,64,0.0001,0.01,15,warp,110,0.746403,0.665864,0.755522,0.504317,0.084226
5,64,0.0005,0.10,15,warp,110,0.742772,0.666781,0.755702,0.504082,0.108786
6,64,0.0010,0.01,15,warp,110,0.717624,0.635345,0.732321,0.503999,0.102213


In [14]:
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)
cal_w, val_w= train_test_split(train_w, 0.8, 22)
cal_i, val_i= train_test_split(train_i, 0.8, 22)

In [ ]:
space_FM_warp = {'factors' : [200], 'regularization' : [0.0005], 'learning_rate' : [0.05], 'iterations' : [30], 'loss' : ['warp'], 'max_sampled' : [100]}
hyper_fm = cv.hyperp_tuning_simple(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp, model_class='FM', item_features=itemf)